In [2]:
import pandas as pd
from nltk.corpus import stopwords
import re
import nltk
from gensim.models.word2vec import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


# Veri Kümemizi Ekleyelim : 

In [2]:
tweets = pd.read_csv("3000tweet/3000tweet.csv", sep=';', encoding="utf8")
tweets.head()

,tweet,durum
0,dun Turkcelle tepkilerimizden sonra bugün Turk...,1
1,girmezmiyim.. Turkcell kartim bile var.. Yarin...,1
2,tam tünelden gecerken 3g cekiyordu :D türkcell...,1
3,turkcell superonline fiber internet veya ADSL ...,1
4,bence Gnçtrkcll Ark Winterfest 2012'de 1.olur ...,1


# Veri Kümemizi Düzenleyelim : 

### NULL Değerleri Silme İşlemi : 

In [3]:
tweets.isnull().sum()

tweet    1
durum    0
dtype: int64

In [4]:
tweets = tweets.dropna().reset_index(drop=True)
tweets.isnull().sum()

tweet    0
durum    0
dtype: int64

### Gereksiz Kelimeleri - Boşukları - İşaretleri Silme İşlemleri :  

In [5]:
stop_word_list = stopwords.words('turkish')
stop_word_list

['acaba',
 'ama',
 'aslında',
 'az',
 'bazı',
 'belki',
 'biri',
 'birkaç',
 'birşey',
 'biz',
 'bu',
 'çok',
 'çünkü',
 'da',
 'daha',
 'de',
 'defa',
 'diye',
 'eğer',
 'en',
 'gibi',
 'hem',
 'hep',
 'hepsi',
 'her',
 'hiç',
 'için',
 'ile',
 'ise',
 'kez',
 'ki',
 'kim',
 'mı',
 'mi',
 'mu',
 'mü',
 'nasıl',
 'ne',
 'neden',
 'nerde',
 'nerede',
 'nereye',
 'niçin',
 'niye',
 'o',
 'sanki',
 'şey',
 'siz',
 'şu',
 'tüm',
 've',
 'veya',
 'ya',
 'yani']

In [6]:
def preprocess_text(sen):
    
    # Sayıları Silme İşlemi
    sentence = re.sub('[\d\s]', ' ', str(sen))

    # Noktalama İşaretlerini Silme İşlemi
    sentence = re.sub('[^\w\s]', ' ', str(sentence))
    
    # Tek Karakterleri Silme İşlemi
    sentence = re.sub(r"\b[\w\s]\b", ' ', str(sentence))
    
    # Birden Çok Boşluğu Silme İşlemi
    sentence = re.sub(r'\s+', ' ', sentence)
        
    # Engellenecek Kelimeleri Silme İşlemi
    WPT = nltk.WordPunctTokenizer()
    tokens = WPT.tokenize(sentence)
    filtered_tokens = [token for token in tokens if token not in stop_word_list]
    single_doc = ' '.join(filtered_tokens)
    
    # Tüm Harfler Küçük Harfe Dönüştürülüyor
    return single_doc.lower()

In [7]:
x = tweets['tweet']
y = tweets['durum']

In [8]:
x = x.apply(preprocess_text)
pd.DataFrame(data=x)

,tweet
0,dun turkcelle tepkilerimizden sonra bugün turk...
1,girmezmiyim turkcell kartim bile var yarindan ...
2,tam tünelden gecerken cekiyordu türkcell in re...
3,turkcell superonline fiber internet adsl sabit...
4,bence gnçtrkcll ark winterfest olur gelir
...,...
2994,evet kesinlikle turkcell konuda an uygun değil...
2995,selocan nedir abicim
2996,özturkcell dönücz korkuyorum çanakkale holding...
2997,turkcell se yi ara puk kodunu ogrenme menusund...


### Kelimeleri Ayıralım ve Kelime Hazinesi Oluşturalım : 

In [9]:
words = []
for i in x:
    words.append(i.split())

print(words[0])

['dun', 'turkcelle', 'tepkilerimizden', 'sonra', 'bugün', 'turkcell', 'twittera', 'sponsor', 'olmuş', 'ne', 'tesadüf', 'değil']


### Kelimeleri Vectörlere Dönüştürelim : 

Veri setimizin işlenebilmesi için text verileri numaralara çevirmemiz gerekir.

In [10]:
token = Tokenizer()
# Her kelimenin sıklığını hesaplar
token.fit_on_texts(x)
# Tüm kelimeleri sayı dizisine dönüştürür
x = token.texts_to_sequences(x)
# Bütün metinlerimiz en uzun kelimeden oluşan metnimiz kadar sütundan oluşan (23) bir dizi ile temsil edilecek.
# 23'den kısa metinlerin boşlukları 0'lar ile doldurulacak.
x = pad_sequences(x)

In [11]:
pd.DataFrame(x).sample(5)

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
1597,0,0,0,0,0,0,0,0,0,0,...,7611,7612,7613,12,1,510,507,43,683,9
228,0,0,0,0,0,0,0,279,3701,3702,...,1,20,267,482,21,175,464,984,9,1935
652,0,0,0,0,0,0,0,0,0,0,...,0,251,1,50,24,4706,2,2210,4707,1251
434,0,0,0,0,0,0,0,0,0,4164,...,31,757,26,395,4165,324,2067,85,4166,490
2066,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,255,3007,1209,9,2783,9038


### Vectörlere Ölçeklendirelim : 

**Verileri Ölçeklendirme Nedir?**

Ölçeklendirme dediğimiz şey veri setindeki iki farklı özelliğin (değişkenin) değerlerini en doğru şekilde analiz edebilmemize olanak sağlayan bir yöntemdir.

- Verilerinizin dağılımının ortalama değeri 0 ve standart sapma değeri 1 olacak şekilde değiştiriyoruz (Standartlaştırma).
- Tutarlılık oluşuyor. Bir veri diğer verilerden daha büyükse diğerleri üzerinde egemenlik kurabilir. Bunu istemeyiz.
- Bir öğrenme modeliyle çalışırken, özellikleri sıfır merkezli bir aralığa ölçeklemek önemlidir.
- Değerleri karşılaştırmak yerine oranları karşılaştırmış oluruz.

**Örnek :**

İki farklı insan düşünelim. Bu insanların birisinin geliri ayda 5.000 Türk Lirası iken diğerinin geliri ayda 200.000 Türk Lirası olsun. Bu insanların ikisi de her ay belirli miktarda maddi yardım için maaşlarının bir kısmını ayırıyorlar. Diyelim ki 5.000 Türk Lirası maaşı olan şahıs 1.500 Türk Lirasını yardıma ayırıyorken, 200.000 Türk Lirası alan şahıs 10.000 Türk Lirasını yardım için ayırıyor.

Normal bir gözle bakarsak 10.000 Türk Lirası yardımda bulunan kişinin daha fazla yardım yaptığını görürüz. Fakat bunu oranladığımızda 5.000 Türk Lirası alan kişinin oransal olarak kat kat fazla yardım yaptığı anlaşılacaktır. İşte özellik ölçeklendirme ile biz bu özelliklerin sayısal değerlerinin birbirine yakın olmasını sağlayıp aralarındaki değerleri karşılaştırmak yerine oranları karşılaştırır.

In [12]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [13]:
pd.DataFrame(x).sample(5)

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
2408,-0.018317,-0.022986,-0.033492,-0.042425,-0.075203,-0.116017,-0.162864,-0.220584,-0.277817,-0.318827,...,-0.480494,-0.514683,-0.553587,-0.597219,-0.627740,-0.664739,-0.697573,-0.717569,-0.801129,1.926648
1728,-0.018317,-0.022986,-0.033492,-0.042425,-0.075203,-0.116017,-0.162864,-0.220584,-0.277817,-0.299339,...,0.458902,2.355422,-0.540730,2.016367,-0.292579,-0.001104,0.175761,1.632382,-0.794440,1.391183
2898,-0.018317,-0.022986,-0.033492,-0.042425,-0.075203,-0.116017,-0.162864,-0.220584,-0.277817,-0.318827,...,-0.480494,-0.514683,-0.553587,-0.597219,-0.627740,-0.664739,-0.697573,-0.717569,-0.801129,-0.846318
222,-0.018317,-0.022986,-0.033492,-0.042425,-0.075203,-0.116017,3.123124,2.282152,0.728759,-0.296415,...,-0.307533,-0.045192,0.695991,-0.278236,-0.617928,-0.602103,-0.110797,-0.717277,-0.799457,-0.843816
61,-0.018317,-0.022986,-0.033492,-0.042425,-0.075203,-0.116017,-0.162864,-0.220584,-0.277817,-0.318827,...,-0.480494,-0.514683,-0.553587,-0.597219,-0.627740,-0.664739,-0.666610,-0.584112,-0.800293,-0.719263


### Y Verilerimizi One Hot Encoding Uygulamamız Gerekiyor :  

**One Hot Encoding Nedir?**

One Hot Encoding, kategorik değişkenlerin ikili (binary) olarak temsil edilmesi anlamına gelmektedir. Bu işlem, ağın model için problemi daha kolay hale getirmesine yardımcı olabilir. 


- Bu işlem ilk önce kategorik değerlerin tamsayı değerleriyle eşlenmesini gerektirir.
- Daha sonra, her bir tamsayı değeri, 1 ile işaretlenmiş tamsayı indeksi dışındaki tüm değerleri   sıfır  olan bir ikili vektör olarak temsil edilir.

In [14]:
pd.DataFrame(y).sample(5)

,durum
1221,2
379,1
103,1
819,2
1639,2


In [15]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

In [16]:
pd.DataFrame(y).sample(5)

,0
2951,2
946,1
160,0
537,0
2354,2


In [17]:
y = to_categorical(y)

In [18]:
pd.DataFrame(y).sample(5)

,0,1,2
534,1.0,0.0,0.0
1209,0.0,1.0,0.0
759,0.0,1.0,0.0
1992,0.0,1.0,0.0
1352,0.0,1.0,0.0


# Train ve Test İşlemleri İçin Ayrımı Gerçekleştirelim : 

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [20]:
print('x Train : ' + str(x_train.shape))
print('x Test : ' + str(x_test.shape))
print('y Train : ' + str(y_train.shape))
print('y Test : ' + str(y_test.shape))

x Train : (2399, 23)
x Test : (600, 23)
y Train : (2399, 3)
y Test : (600, 3)
